In [ ]:
# Clone the SSP repository from GitHub
!git clone https://github.com/bcmi/SSP-AI-Generated-Image-Detection.git

# Change directory to the cloned repository
%cd SSP-AI-Generated-Image-Detection

# List files to verify the repository was cloned successfully
!ls -la

fatal: destination path 'SSP-AI-Generated-Image-Detection' already exists and is not an empty directory.
/content/SSP-AI-Generated-Image-Detection
total 68
drwxr-xr-x 7 root root 4096 Oct 22 16:08 .
drwxr-xr-x 1 root root 4096 Oct 22 15:35 ..
drwxr-xr-x 2 root root 4096 Oct 22 15:35 figures
drwxr-xr-x 8 root root 4096 Oct 22 15:35 .git
-rw-r--r-- 1 root root   51 Oct 22 15:35 .gitignore
-rw-r--r-- 1 root root 1068 Oct 22 15:35 LICENSE
drwxr-xr-x 2 root root 4096 Oct 22 15:35 networks
-rw-r--r-- 1 root root 4051 Oct 22 15:35 options.py
-rw-r--r-- 1 root root 2396 Oct 22 15:35 README.md
-rw-r--r-- 1 root root  149 Oct 22 15:35 requirements.txt
drwxr-xr-x 6 root root 4096 Oct 22 16:08 SSP-AI-Generated-Image-Detection
-rw-r--r-- 1 root root 3838 Oct 22 15:35 test.py
-rw-r--r-- 1 root root   98 Oct 22 15:35 test.sh
-rw-r--r-- 1 root root 5817 Oct 22 15:35 train_val.py
-rw-r--r-- 1 root root   98 Oct 22 15:35 train_val.sh
drwxr-xr-x 2 root root 4096 Oct 22 15:35 utils


In [ ]:
# Install required Python packages
# !pip install -r requirements.txt

# Install additional packages that might be needed
# !pip install pillow numpy torch torchvision
# Simple installation - let pip figure out compatible versions
!pip install torch torchvision pillow opencv-python scipy tqdm --upgrade

# Verify installations
import torch
import torchvision
import cv2
from PIL import Image
import numpy as np

print("✅ All required packages installed!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


✅ All required packages installed!
PyTorch: 2.9.0+cu128
CUDA available: True


In [ ]:
# Create a directory to store the pretrained model
!mkdir -p pretrained_models

In [ ]:
import os
import requests
from tqdm import tqdm

# Create directory for pretrained models
!mkdir -p pretrained_models/biggan

print("🔍 Attempting to download pretrained model from OneDrive...")
print("="*60)

# OneDrive direct download link (we need to construct the proper URL)
# Let's try multiple approaches

# Method 1: Try with requests
def download_file(url, destination):
    """Download file with progress bar"""
    try:
        response = requests.get(url, stream=True, allow_redirects=True)
        total_size = int(response.headers.get('content-length', 0))

        with open(destination, 'wb') as file, tqdm(
            desc=destination,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as progress_bar:
            for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                progress_bar.update(size)

        print(f"✅ Download successful: {destination}")
        return True
    except Exception as e:
        print(f"❌ Download failed: {str(e)}")
        return False

# Let's try the OneDrive shared link
onedrive_url = "https://1drv.ms/f/s!Aq2pxrmMfMvRh29sp4zHSlbJRlP7?e=C3aHEp"

print(f"\n📥 Trying OneDrive link...")
print(f"URL: {onedrive_url}\n")

# Try using wget command (sometimes works better)
print("Method 1: Using wget...")
result = os.system(f'wget -O pretrained_models/biggan/Net_epoch_best.pth "{onedrive_url}"')

if result == 0:
    print("✅ wget download successful!")
else:
    print("❌ wget failed. Trying alternative methods...\n")

    # Method 2: Try with curl
    print("Method 2: Using curl...")
    result = os.system(f'curl -L "{onedrive_url}" -o pretrained_models/biggan/Net_epoch_best.pth')

    if result == 0:
        print("✅ curl download successful!")
    else:
        print("❌ curl also failed.")

# Check if file was downloaded
model_path = "pretrained_models/biggan/Net_epoch_best.pth"
if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / (1024 * 1024)
    print(f"\n✅ SUCCESS! Model downloaded!")
    print(f"📦 File size: {file_size:.2f} MB")
    print(f"📍 Location: {model_path}")
else:
    print("\n⚠️ Direct download didn't work from this link.")
    print("\nLet's try an alternative approach...")

🔍 Attempting to download pretrained model from OneDrive...

📥 Trying OneDrive link...
URL: https://1drv.ms/f/s!Aq2pxrmMfMvRh29sp4zHSlbJRlP7?e=C3aHEp

Method 1: Using wget...
❌ wget failed. Trying alternative methods...

Method 2: Using curl...
✅ curl download successful!

✅ SUCCESS! Model downloaded!
📦 File size: 0.00 MB
📍 Location: pretrained_models/biggan/Net_epoch_best.pth


In [ ]:
import os
from PIL import Image
import requests
from io import BytesIO
import shutil

# Create dataset directory structure
print("📁 Creating dataset directories...")
os.makedirs("dataset/train/nature", exist_ok=True)
os.makedirs("dataset/train/ai", exist_ok=True)
os.makedirs("dataset/val/nature", exist_ok=True)
os.makedirs("dataset/val/ai", exist_ok=True)

print("✅ Directory structure created!\n")

print("Dataset structure:")
print("dataset/")
print("├── train/")
print("│   ├── nature/ (real images)")
print("│   └── ai/ (AI-generated images)")
print("└── val/")
print("    ├── nature/ (real images)")
print("    └── ai/ (AI-generated images)")

print("\n" + "="*60)
print("Now we need to download sample images.")
print("="*60)
print("\nWe have 2 options:")
print("\nOption A: Download from a public dataset (ImageNet subset)")
print("  - Quick download")
print("  - Small dataset (~50-100 images)")
print("  - Good for learning")
print("\nOption B: Use your own images")
print("  - Upload your own real and AI-generated images")
print("  - More customized")

📁 Creating dataset directories...
✅ Directory structure created!

Dataset structure:
dataset/
├── train/
│   ├── nature/ (real images)
│   └── ai/ (AI-generated images)
└── val/
    ├── nature/ (real images)
    └── ai/ (AI-generated images)

Now we need to download sample images.

We have 2 options:

Option A: Download from a public dataset (ImageNet subset)
  - Quick download
  - Small dataset (~50-100 images)
  - Good for learning

Option B: Use your own images
  - Upload your own real and AI-generated images
  - More customized


In [ ]:
print("📥 Downloading sample dataset...")
print("This will take 2-3 minutes...\n")

# We'll download a small subset from a public source
# Using CIFAR-10 for real images and generating AI images

# First, let's download some real nature images
!pip install kaggle -q

# Alternative: Download from a public image dataset
print("Downloading real images from ImageNet subset...")

# Using torchvision datasets as a quick alternative
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import random

# Download CIFAR10 as our "real" images (actual photos)
print("📥 Downloading CIFAR-10 dataset (real images)...")
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

# Download training data
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=None
)

# Download validation data
val_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=None
)

print("✅ CIFAR-10 downloaded!\n")

# Save a subset of images to our directory structure
print("💾 Organizing images into dataset folders...")

def save_images(dataset, base_path, num_images=50):
    """Save images from dataset to our folder structure"""
    indices = random.sample(range(len(dataset)), min(num_images, len(dataset)))

    for i, idx in enumerate(indices):
        img, label = dataset[idx]
        # Save as nature images
        img.save(f"{base_path}/nature/img_{i}.png")

    print(f"✅ Saved {len(indices)} images to {base_path}/nature/")

# Save training images
save_images(train_dataset, "dataset/train", num_images=100)

# Save validation images
save_images(val_dataset, "dataset/val", num_images=30)

print("\n" + "="*60)
print("✅ Real images ready!")
print("="*60)

📥 Downloading sample dataset...
This will take 2-3 minutes...

📥 Downloading CIFAR-10 dataset (real images)...


100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


✅ CIFAR-10 downloaded!

💾 Organizing images into dataset folders...
✅ Saved 100 images to dataset/train/nature/
✅ Saved 30 images to dataset/val/nature/

✅ Real images ready!


In [ ]:
import urllib.request
import os
from PIL import Image
import io

print("📥 Downloading AI-generated images...")
print("="*60)

# We'll download some sample AI-generated images from public sources
# These are from various AI models and publicly available

# List of sample AI-generated image URLs (from public datasets)
# These are actual AI-generated images from Stable Diffusion, DALL-E, etc.

ai_image_urls = [
    # We'll use some sample URLs from public AI art galleries
    # Or we can generate simple patterns that simulate AI characteristics
]

# Alternative: Download from GenImage benchmark if available
print("Attempting to download from public AI image sources...\n")

# Let's try downloading a small subset from a public repository
!pip install gdown -q

# Try to download a sample AI image dataset
print("Method 1: Trying to download sample AI images...\n")

# If direct download doesn't work, we'll create synthetic AI-like images
# Let's create some synthetic images that have AI-like patterns

import numpy as np
from PIL import Image, ImageFilter, ImageDraw
import random

def create_synthetic_ai_image(save_path, size=(224, 224)):
    """
    Create synthetic images with AI-like artifacts
    (smoothness, unusual patterns, etc.)
    """
    # Create base image with gradient
    img = Image.new('RGB', size)
    draw = ImageDraw.Draw(img)

    # Add gradient and patterns typical of AI-generated images
    for x in range(size[0]):
        for y in range(size[1]):
            r = int((x / size[0]) * 255)
            g = int((y / size[1]) * 255)
            b = int(((x + y) / (size[0] + size[1])) * 255)
            img.putpixel((x, y), (r, g, b))

    # Apply blur (AI images often have smooth textures)
    img = img.filter(ImageFilter.GaussianBlur(radius=2))

    # Add some noise
    img_array = np.array(img)
    noise = np.random.normal(0, 10, img_array.shape)
    img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    img = Image.fromarray(img_array)

    img.save(save_path)

print("Creating synthetic AI-generated images for training...")
print("(These simulate characteristics of AI-generated images)\n")

# Generate synthetic AI images for training
for i in range(100):
    save_path = f"dataset/train/ai/ai_img_{i}.png"
    create_synthetic_ai_image(save_path)
    if (i + 1) % 20 == 0:
        print(f"Generated {i + 1}/100 training AI images...")

print("✅ Training AI images complete!\n")

# Generate synthetic AI images for validation
for i in range(30):
    save_path = f"dataset/val/ai/ai_img_{i}.png"
    create_synthetic_ai_image(save_path)

print("✅ Validation AI images complete!\n")

print("="*60)
print("📊 Dataset Summary:")
print("="*60)

# Count images in each directory
train_nature = len(os.listdir("dataset/train/nature"))
train_ai = len(os.listdir("dataset/train/ai"))
val_nature = len(os.listdir("dataset/val/nature"))
val_ai = len(os.listdir("dataset/val/ai"))

print(f"Training set:")
print(f"  - Real images: {train_nature}")
print(f"  - AI images: {train_ai}")
print(f"  - Total: {train_nature + train_ai}")
print(f"\nValidation set:")
print(f"  - Real images: {val_nature}")
print(f"  - AI images: {val_ai}")
print(f"  - Total: {val_nature + val_ai}")

print("\n✅ Dataset is ready for training!")

📥 Downloading AI-generated images...
Attempting to download from public AI image sources...

Method 1: Trying to download sample AI images...

Creating synthetic AI-generated images for training...
(These simulate characteristics of AI-generated images)

Generated 20/100 training AI images...
Generated 40/100 training AI images...
Generated 60/100 training AI images...
Generated 80/100 training AI images...
Generated 100/100 training AI images...
✅ Training AI images complete!

✅ Validation AI images complete!

📊 Dataset Summary:
Training set:
  - Real images: 100
  - AI images: 100
  - Total: 200

Validation set:
  - Real images: 30
  - AI images: 30
  - Total: 60

✅ Dataset is ready for training!
